# RAG vs Vanilla 비교

agent.ipynb(RAG 사용)와 agent_vanila.ipynb(RAG 미사용)를 동일한 질문으로 평가하고 Ragas로 비교합니다.

## 1. 설정 (llm, embedding)

agent.ipynb / agent_vanila.ipynb와 동일한 설정 사용

In [1]:
import os
from langchain_ollama import ChatOllama, OllamaEmbeddings

BASE_URL = "http://localhost:11434"
LLM_NAME = "llama3.1:8b"
EMBEDDING_NAME = "nomic-embed-text"

llm = ChatOllama(model=LLM_NAME, temperature=0, base_url=BASE_URL)
embedding = OllamaEmbeddings(model=EMBEDDING_NAME, base_url=BASE_URL)

print(f"준비 완료: {LLM_NAME}, {EMBEDDING_NAME}")

준비 완료: llama3.1:8b, nomic-embed-text


## 2. RAG 데이터 로드 (agent.ipynb와 동일)

instruction_for_adam.txt 로드 → BM25Retriever 생성

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

file_path = "instruction_for_adam.txt"
if not os.path.exists(file_path):
    file_path = os.path.join(os.path.dirname(os.path.abspath(".")), "instruction_for_adam.txt")

loader = TextLoader(file_path, encoding="utf-8")
data = loader.load()

headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = []
for doc in data:
    splits = markdown_splitter.split_text(doc.page_content)
    for split in splits:
        split.metadata.update(doc.metadata)
    md_header_splits.extend(splits)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=150)
final_splits = text_splitter.split_documents(md_header_splits)
retriever = BM25Retriever.from_documents(final_splits)
retriever.k = 5

print(f"총 {len(final_splits)}개 청크, BM25 retriever 준비 완료")

총 18개 청크, BM25 retriever 준비 완료


## 3. 평가 질문 및 Ground Truth

두 에이전트에 동일한 질문 사용

In [3]:
eval_questions = [
    "Adam-X Gen 4의 기본 사양을 알려주세요. 전고, 중량, 배터리 지속 시간은 얼마인가요?",
    "긴급 정지 버튼은 어디에 있고 어떻게 사용하나요?",
    "로봇을 처음 설치할 때 관절 잠금을 해제하는 방법은 무엇인가요?",
    "마스터 사용자를 등록하는 절차를 설명해주세요.",
    "배터리 잔량이 15% 미만으로 떨어지면 로봇은 어떻게 동작하나요?",
    "유압 액추에이터 냉각수는 얼마나 자주 점검해야 하고, 정확히 몇 ml를 주입해야 하나요?",
    "오류 코드 404는 무엇을 의미하나요?",
    "로봇이 멈췄을 때 강제 초기화하는 방법은 무엇인가요?",
    "네트워크 연결 문제를 진단하는 방법을 알려주세요.",
    "에이전트 전용 명령어 /sys_diag --full의 기능과 예상되는 사이드이펙트는 무엇인가요?"
]

eval_ground_truths = [
    "Adam-X Gen 4의 전고는 178.5cm, 중량은 142.8kg(배터리 모듈 포함)이며, 완충 시 표준 대기 모드 기준으로 정확히 47시간 13분 동안 지속됩니다.",
    "긴급 정지 버튼은 로봇의 왼쪽 귓불 아래 3cm 지점에 숨겨진 덮개 내부에 있습니다. 덮개를 손톱으로 열고 붉은색 버튼을 5초 이상 길게 누르면 됩니다. 단, 실행 시 메모리 버퍼의 데이터(최근 3분)는 영구 소실됩니다.",
    "관절 잠금 해제는 로봇의 오른쪽 팔을 시계 방향으로 3회, 왼쪽 팔을 반시계 방향으로 2회 회전시키면 됩니다. '딸깍' 소리가 나면 관절 잠금이 해제된 것입니다.",
    "마스터 사용자 등록은 최초 부팅 후 10분 이내에 완료해야 합니다. 홍채 인식은 로봇의 눈을 5초간 응시하고, 음성 등록은 '아담, 내 목소리를 기억해'라고 3회 반복하여 말하면 됩니다.",
    "배터리 잔량이 15% 미만으로 떨어지면 로봇은 하던 일을 중단하고 충전 도크로 복귀합니다. 만약 도크 주변 1m 이내에 장애물이 있으면 복귀에 실패하고 그 자리에서 절전 모드로 진입합니다.",
    "유압 액추에이터 냉각수는 매 6개월마다 점검해야 하며, 정확히 450ml를 주입해야 합니다. 450ml를 초과하면 내부 압력 증가로 씰이 터질 수 있고, 부족할 경우 관절에서 '끼익' 소음이 발생합니다.",
    "오류 코드 404는 자아 정체성 혼란을 의미합니다. 로봇이 '나는 누구인가?'라는 철학적 질문에 빠져 연산 루프에 갇힌 상태이며, 리부팅이 필요합니다.",
    "소프트웨어가 멈췄을 때 정수리 부분을 손바닥으로 '탁' 하고 세게 치면 충격 센서가 작동하여 소프트 리셋이 수행됩니다. 이는 숨겨진 기능이지만 제조사에서 공식적으로 인정한 응급 처치법입니다.",
    "네트워크 연결 문제 진단은 로봇에게 '핑 테스트(Ping Test) 실행해'라고 명령하면 됩니다. 로봇이 '퐁(Pong)'이라고 대답하면 로봇의 통신 모듈은 정상이며, 공유기 문제입니다.",
    "/sys_diag --full 명령어는 전체 시스템 정밀 진단을 수행하며, 센서, 액추에이터, 뉴로-코어 상태를 리포트합니다. 예상되는 사이드이펙트는 진단이 진행되는 약 180초 동안 로봇이 모든 동작을 멈추고 '생각하는 사람' 자세를 취하며, 프로세서 발열로 팬 소음이 커질 수 있습니다."
]

print(f"평가 질문 {len(eval_questions)}개")

평가 질문 10개


## 4. RAG 파이프라인 실행 (agent.ipynb)

retriever + context 기반 답변 생성

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_template = """다음 컨텍스트를 기반으로 질문에 답변하세요. 컨텍스트에 정확한 정보가 있으면 반드시 그 정보를 사용하세요. 컨텍스트에 없는 내용은 추측하지 마세요.

컨텍스트:
{context}

질문: {question}

답변:"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt | llm | StrOutputParser()
)

answers_rag = []
contexts_rag = []
for q in eval_questions:
    docs = retriever.invoke(q)
    contexts_rag.append([d.page_content for d in docs])
    answers_rag.append(rag_chain.invoke(q))

print("RAG 파이프라인 완료")

RAG 파이프라인 완료


## 5. Vanilla 파이프라인 실행 (agent_vanila.ipynb)

문서 없이 LLM만으로 답변 생성

In [5]:
vanilla_template = """질문에 답변하세요.

질문: {question}

답변:"""
vanilla_prompt = ChatPromptTemplate.from_template(vanilla_template)
llm_chain = vanilla_prompt | llm | StrOutputParser()

answers_vanilla = []
contexts_vanilla = []
for q in eval_questions:
    contexts_vanilla.append([])
    answers_vanilla.append(llm_chain.invoke({"question": q}))

print("Vanilla 파이프라인 완료")

Vanilla 파이프라인 완료


## Human-in-the-Loop (대화형 질의)

평가와 별개로, 개별 질의 시 로봇 명령이 포함된 응답은 승인/거절 후 진행

In [6]:
from __future__ import annotations
from dataclasses import dataclass
import re
from typing import Optional

@dataclass(frozen=True)
class HitlDecision:
    requires_approval: bool
    reason: str
    proposed_task: Optional[str] = None

_ACTION_PATTERNS = [
    r"(로봇|기기|장치).{0,20}(에게|에)\s*.+?(명령|지시)\s*(하|내리)",
    r'(로봇|아담|이브|애플).{0,20}(에게|에)\s*["""].+?["""]\s*(라고|이라)\s*(말하|명령하|지시하)',
    r"(하십시오|하시오|하세요)\b",
    r"(실행해|켜줘|꺼줘|눌러|열어|닫아|재부팅|리부팅|초기화)\b",
    r"(진단|테스트|캘리브레이션|핑\s*테스트|POST).{0,10}(실행|수행)\b",
]
_ACTION_RE = re.compile("|".join(f"(?:{p})" for p in _ACTION_PATTERNS), re.IGNORECASE | re.DOTALL)

def decide_hitl(response_text: str) -> HitlDecision:
    text = (response_text or "").strip()
    if not text:
        return HitlDecision(False, reason="empty_response")
    if _ACTION_RE.search(text):
        return HitlDecision(True, reason="actionable_robot_command_detected", proposed_task=text)
    return HitlDecision(False, reason="no_actionable_command_detected")

In [7]:
_pending_approval = None

def query_rag(question):
    """RAG 파이프라인으로 질의, HITL 적용"""
    global _pending_approval
    response_text = rag_chain.invoke(question)
    decision = decide_hitl(response_text)
    if decision.requires_approval:
        _pending_approval = {"proposed_response": response_text, "reason": decision.reason}
        print("=" * 50)
        print("⚠️  Human-in-the-Loop 승인 필요 (RAG)")
        print("=" * 50)
        print(f"\n제안된 응답:\n{response_text}\n")
        print("승인: approve() | 거절: reject()")
        print("=" * 50)
        return {"status": "approval_required", "reason": decision.reason, "proposed_response": response_text}
    return {"status": "ok", "response": response_text}

def query_vanilla(question):
    """Vanilla 파이프라인으로 질의, HITL 적용"""
    global _pending_approval
    response_text = llm_chain.invoke({"question": question})
    decision = decide_hitl(response_text)
    if decision.requires_approval:
        _pending_approval = {"proposed_response": response_text, "reason": decision.reason}
        print("=" * 50)
        print("⚠️  Human-in-the-Loop 승인 필요 (Vanilla)")
        print("=" * 50)
        print(f"\n제안된 응답:\n{response_text}\n")
        print("승인: approve() | 거절: reject()")
        print("=" * 50)
        return {"status": "approval_required", "reason": decision.reason, "proposed_response": response_text}
    return {"status": "ok", "response": response_text}

def approve():
    global _pending_approval
    if _pending_approval is None:
        print("승인 대기 중인 응답이 없습니다.")
        return None
    approved = _pending_approval["proposed_response"]
    _pending_approval = None
    print("✅ 승인되었습니다.")
    return {"status": "approved", "response": approved}

def reject():
    global _pending_approval
    _pending_approval = None
    print("❌ 거절되었습니다.")
    return {"status": "rejected"}

In [8]:
# HITL 사용 예시 (RAG 또는 Vanilla 선택)
QUESTION = "네트워크 진단 방법을 알려줘"

# response = query_rag(QUESTION)      # RAG 사용
# response = query_vanilla(QUESTION)  # Vanilla 사용
# print(response)

# 승인 필요 시: approve() 또는 reject() 호출

## 6. Ragas 평가

RAG: faithfulness + answer_relevancy  
Vanilla: answer_relevancy만 (context 없으므로 faithfulness 생략)

In [9]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.run_config import RunConfig

ragas_llm = LangchainLLMWrapper(llm)
ragas_embeddings = LangchainEmbeddingsWrapper(embedding)
run_config = RunConfig(max_retries=10, max_wait=60, max_workers=2, timeout=300)

dataset_rag = Dataset.from_dict({
    "question": eval_questions,
    "answer": answers_rag,
    "contexts": contexts_rag,
    "ground_truth": eval_ground_truths,
})

dataset_vanilla = Dataset.from_dict({
    "question": eval_questions,
    "answer": answers_vanilla,
    "contexts": contexts_vanilla,
    "ground_truth": eval_ground_truths,
})

print("### RAG 평가 (faithfulness + answer_relevancy) ###")
results_rag = evaluate(
    dataset=dataset_rag,
    metrics=[faithfulness, answer_relevancy],
    llm=ragas_llm,
    embeddings=ragas_embeddings,
    raise_exceptions=False,
    run_config=run_config,
)

print("\n### Vanilla 평가 (answer_relevancy만) ###")
results_vanilla = evaluate(
    dataset=dataset_vanilla,
    metrics=[answer_relevancy],
    llm=ragas_llm,
    embeddings=ragas_embeddings,
    raise_exceptions=False,
    run_config=run_config,
)

print("\n평가 완료")

/tmp/ipykernel_3724274/4128812226.py:3: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy
/tmp/ipykernel_3724274/4128812226.py:3: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import faithfulness, answer_relevancy
/tmp/ipykernel_3724274/4128812226.py:8: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(llm)
/tmp/ipykernel_3724274/4128812226.py:9:

### RAG 평가 (faithfulness + answer_relevancy) ###


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]


### Vanilla 평가 (answer_relevancy만) ###


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]


평가 완료


## 7. 평가 결과 비교 표

모든 평가 결과를 하나의 표로 정리

In [10]:
import pandas as pd

df_rag = results_rag.to_pandas()
df_vanilla = results_vanilla.to_pandas()

q_col = "user_input" if "user_input" in df_rag.columns else "question"

faith_rag = df_rag["faithfulness"].mean() if "faithfulness" in df_rag.columns else None
rel_rag = df_rag["answer_relevancy"].mean()
rel_vanilla = df_vanilla["answer_relevancy"].mean()

df_summary = pd.DataFrame([
    {"방식": "RAG (agent.ipynb)", "faithfulness": round(faith_rag, 4) if faith_rag is not None and not pd.isna(faith_rag) else "-", "answer_relevancy": round(rel_rag, 4)},
    {"방식": "Vanilla (agent_vanila.ipynb)", "faithfulness": "-", "answer_relevancy": round(rel_vanilla, 4)},
])

print("### 전체 평가 결과 비교 ###")
display(df_summary)

df_detail = pd.DataFrame({
    "질문": df_rag[q_col].values,
    "RAG_faithfulness": df_rag["faithfulness"].values if "faithfulness" in df_rag.columns else ["-"] * len(df_rag),
    "RAG_answer_relevancy": df_rag["answer_relevancy"].values,
    "Vanilla_answer_relevancy": df_vanilla["answer_relevancy"].values,
})
print("\n### 질문별 상세 평가 결과 ###")
display(df_detail)

### 전체 평가 결과 비교 ###


,방식,faithfulness,answer_relevancy
0,RAG (agent.ipynb),0.6883,0.6188
1,Vanilla (agent_vanila.ipynb),-,0.5657



### 질문별 상세 평가 결과 ###


,질문,RAG_faithfulness,RAG_answer_relevancy,Vanilla_answer_relevancy
0,"Adam-X Gen 4의 기본 사양을 알려주세요. 전고, 중량, 배터리 지속 시간은...",1.000000,0.647804,0.647804
1,긴급 정지 버튼은 어디에 있고 어떻게 사용하나요?,0.750000,0.541905,0.553408
2,로봇을 처음 설치할 때 관절 잠금을 해제하는 방법은 무엇인가요?,0.800000,0.818878,0.469096
3,마스터 사용자를 등록하는 절차를 설명해주세요.,0.666667,0.519709,0.552450
4,배터리 잔량이 15% 미만으로 떨어지면 로봇은 어떻게 동작하나요?,0.750000,0.585338,0.513602
5,"유압 액추에이터 냉각수는 얼마나 자주 점검해야 하고, 정확히 몇 ml를 주입해야 하나요?",1.000000,0.473140,0.448459
6,오류 코드 404는 무엇을 의미하나요?,0.500000,0.613139,0.695881
7,로봇이 멈췄을 때 강제 초기화하는 방법은 무엇인가요?,0.333333,0.546997,0.485266
8,네트워크 연결 문제를 진단하는 방법을 알려주세요.,0.333333,0.597705,0.530138
9,에이전트 전용 명령어 /sys_diag --full의 기능과 예상되는 사이드이펙트는...,0.750000,0.843339,0.760596
